In [6]:
pip install PyPDF2


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:

from transformers import pipeline
from PyPDF2 import PdfReader
import re

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_path = "/home/anjana/Project/generator/question_papers/note.pdf"
text = extract_text_from_pdf(pdf_path)

question_generator = pipeline("text2text-generation", model="valhalla/t5-small-qa-qg-hl")

def split_text_into_chunks(text, max_length=512):
    # Split text into sentences
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_length:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

text_chunks = split_text_into_chunks(text)

def generate_questions(text_chunks):
    questions = []
    for chunk in text_chunks:
        # Generate questions using the model
        result = question_generator(chunk, max_length=128, num_return_sequences=1)  # Change 3 to 1
        for res in result:
            questions.append(res['generated_text'])
    return questions


# Generate questions
questions = generate_questions(text_chunks)

# Step 7: Save the Generated Questions to a File
output_file = "generated_questions.txt"
with open(output_file, "w") as f:
    for i, question in enumerate(questions, 1):
        f.write(f"{i}. {question}\n")

print(f"Generated questions saved to {output_file}")

Token indices sequence length is longer than the specified maximum sequence length for this model (556 > 512). Running this sequence through the model will result in indexing errors


Generated questions saved to generated_questions.txt
